เคส

# 0. Magic Commands


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import Library

In [2]:
from fastai import *
from fastai.tabular import *

# 2. ข้อมูล

เคสนี้เราจะใช้ Dataset ตาราง

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.


You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

Files
train.csv - historical data including Sales
test.csv - historical data excluding Sales
sample_submission.csv - a sample submission file in the correct format
store.csv - supplemental information about the stores
Data fields
Most of the fields are self-explanatory. The following are descriptions for those that aren't.

Id - an Id that represents a (Store, Date) duple within the test set
Store - a unique Id for each store
Sales - the turnover for any given day (this is what you are predicting)
Customers - the number of customers on a given day
Open - an indicator for whether the store was open: 0 = closed, 1 = open
StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
StoreType - differentiates between 4 different store models: a, b, c, d
Assortment - describes an assortment level: a = basic, b = extra, c = extended
CompetitionDistance - distance in meters to the nearest competitor store
CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
Promo - indicates whether a store is running a promo on that day
Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store


https://www.kaggle.com/c/rossmann-store-sales

เราจะใช้ข้อมูลภายนอกมาประกอบด้วย

http://files.fast.ai/part2/lesson14/rossmann.tgz

In [3]:
# ! wget http://files.fast.ai/part2/lesson14/rossmann.tgz 
# ! mv ./rossmann.tgz ~/.fastai/data
# ! mkdir ~/.fastai/data/rossmann
# ! tar -xvzf ~/.fastai/data/rossmann.tgz -C ~/.fastai/data/rossmann

In [4]:
path=Config().data_path()/Path('rossmann/')

## 2.1 ดูรายการไฟล์

ls ดูว่ามีไฟล์อะไรบ้าง

In [5]:
path.ls()

[PosixPath('/home/jupyter/.fastai/data/rossmann/state_names.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/joined'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/test.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/sample_submission.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/googletrend.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/weather.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/joined_test'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/store.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/train.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/store_states.csv')]

ใช้ Pandas read_csv ไปใส่ DataFrame

In [6]:
table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend', 'weather', 'test']
tables = [pd.read_csv(path/f'{fname}.csv', low_memory=False) for fname in table_names]
train, store, store_states, state_names, googletrend, weather, test = tables
len(train),len(test)

(1017209, 41088)

## 2.2 ดูข้อมูลทุกตาราง

เป็นการดี ที่เราสำรวจข้อมูลก่อน เพื่อจะได้รู้จักหน้าตาข้อมูลในทุกตาราง

In [7]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [8]:
train.describe(include='all')

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
count,1.017209e+06,1.017209e+06,1017209,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1017209,1.017209e+06
unique,NaN,NaN,942,NaN,NaN,NaN,NaN,4,NaN
top,NaN,NaN,2013-08-26,NaN,NaN,NaN,NaN,0,NaN
freq,NaN,NaN,1115,NaN,NaN,NaN,NaN,986159,NaN
mean,5.584297e+02,3.998341e+00,NaN,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,NaN,1.786467e-01
std,3.219087e+02,1.997391e+00,NaN,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,NaN,3.830564e-01
min,1.000000e+00,1.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00
25%,2.800000e+02,2.000000e+00,NaN,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00
50%,5.580000e+02,4.000000e+00,NaN,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00
75%,8.380000e+02,6.000000e+00,NaN,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,NaN,0.000000e+00


In [9]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [10]:
store_states.head()

,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


In [11]:
state_names.head()

,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB


In [12]:
googletrend.head()

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


In [13]:
weather.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


In [14]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


## 2.3 แปลงชนิดข้อมูล

### train.StateHoliday 

StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None

In [15]:
train.StateHoliday.unique()

array(['0', 'a', 'b', 'c'], dtype=object)

In [16]:
train.StateHoliday.value_counts()

0    986159
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64

แปลงให้เป็น boolean

In [17]:
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'

## 2.4 แตก Field ใหม่

In [18]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

In [19]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

## 2.5 Date Part

In [20]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [21]:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

In [22]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,1,5,2015-07-31,5263,555,1,1,False,1,2015,...,31,4,212,True,False,False,False,False,False,1438300800
1,2,5,2015-07-31,6064,625,1,1,False,1,2015,...,31,4,212,True,False,False,False,False,False,1438300800
2,3,5,2015-07-31,8314,821,1,1,False,1,2015,...,31,4,212,True,False,False,False,False,False,1438300800
3,4,5,2015-07-31,13995,1498,1,1,False,1,2015,...,31,4,212,True,False,False,False,False,False,1438300800
4,5,5,2015-07-31,4822,559,1,1,False,1,2015,...,31,4,212,True,False,False,False,False,False,1438300800


## 2.6 Join ตารางเข้าด้วยกัน

Join ข้อมูลเข้าด้วยกัน เป็นตารางใหญ่ตารางเดียว

In [23]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

In [24]:
weather = join_df(weather, state_names, "file", "StateName")

In [25]:
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [26]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [27]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [28]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [29]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [30]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [31]:
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)

fillna

In [32]:
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

สร้าง Field Date ขึ้นมาจาก 2 Field Number และใช้ Field ใหม่นั้น คำนวน จำนวนวันตั้งแต่คู่แข่งเปิดสาขา

In [33]:
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

Clear ข้อมูล Error

In [34]:

for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

In [35]:
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,  1, 14, 20,  8, 18,  6, 21,  5])

In [36]:
# ! pip install isoweek

In [37]:
from isoweek import Week
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [38]:

for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [39]:
joined.to_pickle(path/'joined')
joined_test.to_pickle(path/'joined_test')

## 2.6 Durations

In [40]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

In [41]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [42]:
#df = train[columns]
df = train[columns].append(test[columns])

In [43]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [44]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [45]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [46]:
df = df.set_index("Date")

In [47]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [48]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

Rolling

In [49]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [50]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

In [51]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [52]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [53]:
df.reset_index(inplace=True)

In [54]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [55]:
df.drop(columns,1,inplace=True)

In [56]:
df.head()

,Date,Store,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,13,0,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,1,12,0,104,0,0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1,11,0,103,0,0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,1,10,0,102,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,1,9,0,101,0,9,-1,0.0,0.0,0.0,0.0,0.0,4.0


In [58]:
df.to_pickle(path/'df')

In [59]:
df["Date"] = pd.to_datetime(df.Date)

In [60]:
df.columns

Index(['Date', 'Store', 'AfterSchoolHoliday', 'BeforeSchoolHoliday',
       'AfterStateHoliday', 'BeforeStateHoliday', 'AfterPromo', 'BeforePromo',
       'SchoolHoliday_bw', 'StateHoliday_bw', 'Promo_bw', 'SchoolHoliday_fw',
       'StateHoliday_fw', 'Promo_fw'],
      dtype='object')

In [62]:
joined = pd.read_pickle(path/'joined')
joined_test = pd.read_pickle(path/f'joined_test')

In [63]:
joined = join_df(joined, df, ['Store', 'Date'])

In [64]:
joined_test = join_df(joined_test, df, ['Store', 'Date'])

In [65]:
joined = joined[joined.Sales!=0]

In [66]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

In [68]:
joined.to_pickle(path/'train_clean')
joined_test.to_pickle(path/'test_clean')

# 3. เตรียมข้อมูล



ss

In [73]:
train_df = pd.read_pickle(path/'train_clean')
test_df = pd.read_pickle(path/'test_clean')

In [74]:
len(train_df),len(test_df)

(844338, 41088)

In [75]:
train_df.head().T

,0,1,2,3,4
index,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,1,1,1,1,1


In [86]:
procs=[FillMissing, Categorify, Normalize]

In [87]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

cont_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

In [88]:
dep_var = 'Sales'
df = train_df[cat_vars + cont_vars + [dep_var,'Date']].copy()

In [89]:
test_df['Date'].min(), test_df['Date'].max()

(Timestamp('2015-08-01 00:00:00'), Timestamp('2015-09-17 00:00:00'))

In [90]:
cut = train_df['Date'][(train_df['Date'] == train_df['Date'][len(test_df)])].index.max()
cut

41395

In [91]:
valid_idx = range(cut)

In [92]:
df[dep_var].head()

0     5263
1     6064
2     8314
3    13995
4     4822
Name: Sales, dtype: int64

สร้าง Databunch จากการกำหนดค่าด้านบน

In [93]:
databunch = (TabularList.from_df(df, path=path, cat_names=cat_vars, cont_names=cont_vars, procs=procs,)
                .split_by_idx(valid_idx)
                .label_from_df(cols=dep_var, label_cls=FloatList, log=True)
                .add_test(TabularList.from_df(test_df, path=path, cat_names=cat_vars, cont_names=cont_vars))
                .databunch())

## สำรวจข้อมูล ##


*เราสามารถรัน cell นี้หลายครั้ง เพื่อเรียกดู batch ต่อ ๆ ไป ได้เรื่อย ๆ เป็นการสำรวจข้อมูล*

In [94]:
databunch.show_batch(rows=10)

Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,PromoInterval,CompetitionOpenSinceYear,Promo2SinceYear,State,Week,Events,Promo_fw,Promo_bw,StateHoliday_fw,StateHoliday_bw,SchoolHoliday_fw,SchoolHoliday_bw,CompetitionDistance_na,CloudCover_na,CompetitionDistance,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday,target
906,3,2013,8,21,False,24,0,a,a,#na#,2010,1900,HE,34,#na#,2.0,2.0,0.0,0.0,0.0,2.0,False,False,-0.6870,1.3360,1.1731,0.8602,-0.7093,-1.2612,-1.4051,-1.0657,-0.6483,0.2552,0.2253,0.3807,1.4154,-0.1397,-0.8994,-0.4796,8.987322
873,3,2014,10,1,False,24,0,a,a,#na#,2008,1900,SN,40,Fog-Rain,5.0,3.0,1.0,0.0,0.0,0.0,False,False,-0.4371,0.6144,1.0322,1.1756,0.8653,0.8088,0.0716,-1.0657,-1.1514,-0.3604,0.2253,0.2726,2.0788,1.1232,1.1119,-0.4796,8.410275
714,5,2014,3,21,False,24,25,d,c,"Jan,Apr,Jul,Oct",2005,2013,BW,12,Rain,1.0,5.0,0.0,0.0,0.0,0.0,False,False,0.8482,0.7346,0.4683,0.0716,-2.9401,-2.1811,-1.7107,1.2646,0.1902,1.4864,-0.7716,-1.1325,1.1310,0.3223,1.1119,-0.4796,8.492491
677,3,2013,1,2,False,0,25,d,a,"Feb,May,Aug,Nov",2014,2009,BY,1,Fog-Rain,2.0,0.0,1.0,1.0,3.0,2.0,False,False,-0.4756,-1.1896,-1.2233,-1.3478,0.8653,1.0387,1.2429,-0.3999,0.1902,0.2552,-0.7716,-0.3759,-1.1752,1.0616,-0.8994,2.0852,8.811801
173,3,2014,5,21,False,17,0,a,a,#na#,2012,1900,BE,21,#na#,3.0,3.0,0.0,0.0,0.0,0.0,False,True,-0.6537,1.6968,1.4551,1.1756,-0.7093,-2.1045,-1.8125,0.1549,-0.9837,0.2552,-0.5904,-0.8083,-0.5750,0.9384,1.1119,-0.4796,9.077152
896,2,2013,1,15,False,4,0,a,c,#na#,2012,1900,SH,3,Fog,1.0,3.0,0.0,0.0,0.0,0.0,False,False,-0.6768,-2.3923,-2.6330,-2.6095,-0.0532,1.2687,1.8030,-1.3986,-1.1514,-0.3604,-0.3185,-1.0245,-0.7645,-1.0638,-0.8994,-0.4796,8.561401
346,5,2013,12,20,False,0,0,a,c,#na#,1900,1900,NW,51,#na#,1.0,5.0,2.0,0.0,4.0,0.0,False,False,0.3382,-0.7085,-0.8004,-0.7169,0.3404,0.5021,0.4281,0.1549,0.3579,-0.9761,0.1347,-0.2678,0.3412,1.0308,1.1119,-0.4796,9.2724695
160,6,2013,5,4,False,24,0,d,c,#na#,2005,1900,HE,18,Fog-Rain-Thunderstorm,0.0,5.0,1.0,1.0,0.0,0.0,False,False,1.4941,0.3738,0.6092,0.8602,0.8653,0.5021,0.3263,-0.1780,-0.6483,0.2552,0.0441,-1.0245,-1.1120,1.0308,-0.8994,-0.4796,8.511175
769,2,2013,9,3,False,0,25,b,b,"Jan,Apr,Jul,Oct",1900,2012,NW,36,#na#,1.0,3.0,0.0,0.0,1.0,5.0,False,False,-0.5909,0.9752,1.0322,1.0179,0.0780,-0.0345,-0.0811,-1.0657,-0.9837,-0.3604,-0.1372,-0.2678,1.8261,0.2607,-0.8994,2.0852,9.125653
98,2,2013,10,22,False,24,25,d,c,"Jan,Apr,Jul,Oct",2006,2012,NW,43,Rain-Thunderstorm,4.0,2.0,0.0,0.0,5.0,2.0,False,False,1.2416,0.9752,1.1731,1.3333,0.0780,-0.0345,0.1735,1.9304,1.8672,0.2552,-0.3185,-0.5921,-0.6066,0.8768,1.1119,2.0852,8.254529


aa

# 4. สร้างโมเดล

เนื่องจากตารางเล็ก ๆ ไม่ซับซ้อน เราจะสร้างโมเดล ที่ประกอบด้วย Dense Layer ที่ Deep แค่ 2 Hidden Layer โดยที่เหลือใช้ค่า Default ทั้งหมด เรื่องชนิดของ Layer เช่น Dense ไว้เราจะอธิบายต่อไป

In [97]:
max_log_y = np.log(np.max(train_df['Sales'])*1.2)
y_range = torch.tensor([0, max_log_y], device=defaults.device)

In [103]:
learner = tabular_learner(databunch, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04, 
                        y_range=y_range, metrics=exp_rmspe, callback_fns=[ShowGraph])

# 5. เริ่มต้นเทรนโมเดล

เทรนด้วยค่า Default ไป 1 Cycle สามารถใช้ Learning Rate สูง ๆ ได้เลย

In [104]:
len(databunch.train_ds.cont_names)

16

In [105]:
learner.fit_one_cycle(5, 1e-1, wd=0.2)

epoch,train_loss,valid_loss,exp_rmspe,time


Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

## สำเร็จแล้ว ##

เพียงแค่เวลา 3 วินาที เราเทรน Model ได้ accuracy ประมาณ 0.84 หรือ ความแม่นยำประมาณ 84%


In [ ]:
learner.save("05b-tabular-rossmann-1")

In [ ]:
learner.load("05b-tabular-rossmann-1");

In [ ]:
learner.fit_one_cycle(5, 3e-4)

In [ ]:
learner.fit_one_cycle(5, 3e-4)

# 6. ดูผลลัพธ์

ลองดูข้อมูล Record ที่ 850

In [ ]:
row = dataframe.iloc[850]
row

ให้โมเดลทำนายกัน

In [ ]:
learner.predict(row)


โมเดลทำนายได้ถูกต้อง

In [ ]:
test_preds=learn.get_preds(DatasetType.Test)
test_df["Sales"]=np.exp(test_preds[0].data).numpy().T[0]
test_df[["Id","Sales"]]=test_df[["Id","Sales"]].astype("int")
test_df[["Id","Sales"]].to_csv("rossmann_submission.csv",index=False)

# 7. สรุป

1. 

# Credit 

* [FastAI: Practical Deep Learning for Coders, v3 - Lesson 6](https://course.fast.ai/videos/?lesson=6)
* [Rossmann Store Sales](https://www.kaggle.com/c/rossmann-store-sales)